# Lab 6: Multi-Agent Orchestration



## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from letta import create_client 

client = create_client()

/Users/azinasgarian/miniconda3/envs/mem/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Letta.letta.server.server - INFO - Creating sqlite engine sqlite:////Users/azinasgarian/.letta/sqlite.db


In [3]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig


# Set the default llm config
client.set_default_llm_config(LLMConfig.default_config("gpt-4"))

# Set the default embedding config
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002"))

## Section 1: Shared Memory Block

In [4]:
from letta.schemas.block import Block

org_description= "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

org_block = Block(name="company",
                  label="Company Description",
                  value=org_description )

In [5]:
org_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=5000, template_name='company', is_template=False, label='Company Description', description=None, metadata={}, id='block-43df646d-5cf2-411a-90f6-ed6ab855d4db', organization_id=None, created_by_id=None, last_updated_by_id=None)

In [6]:
from letta.schemas.memory import BasicBlockMemory

class OrgMemory(BasicBlockMemory): 

    def __init__(self, persona: str, org_block: Block): 
        persona_block = Block(name="persona", label="Agent Persona", value=persona)
        super().__init__(blocks=[persona_block, org_block])

In [7]:
#cleanup. This code will remove agents if the code is run more than once otherwise it will do nothing.
for agent in client.list_agents(): 
    client.delete_agent(agent.id)

In [8]:
# client.delete_agent(recruiter_agent.id)  #cleanup code for repeated runs if needed

## Section 2: Orchestrating Multiple Agents 

#### Evaluator Agent

In [9]:
def read_resume(name: str): 
    """
    Read the resume data for a candidate given the name

    Args: 
        name (str): Candidate name 

    Returns: 
        resume_data (str): Candidate's resume data 
    """
    import os
    filepath = os.path.join("data", "resumes", name.lower().replace(" ", "_") + ".txt")
    #print("read", filepath)
    return open(filepath).read()

In [10]:
def submit_evaluation( 
    candidate_name: str, 
    reach_out: bool, 
    resume: str, 
    justification: str
): 
    """
    Submit a candidate for outreach. 

    Args: 
        candidate_name (str): The name of the candidate
        reach_out (bool): Whether to reach out to the candidate
        resume (str): The text representation of the candidate's resume 
        justification (str): Justification for reaching out or not
    """
    from letta import create_client 
    client = create_client()

    message = "Reach out to the following candidate. " \
    + f"Name: {candidate_name}\n" \
    + f"Resume Data: {resume}\n" \
    + f"Justification: {justification}"
    print("eval agent", candidate_name)
    if reach_out:
        response = client.send_message(
            agent_name="outreach_agent", 
            role="user", 
            message=message
        ) 
    else: 
        print(f"Candidate {candidate_name} is rejected: {justification}")

In [11]:
read_resume_tool = client.create_tool(read_resume) 
submit_evaluation_tool = client.create_tool(submit_evaluation)

In [12]:
skills = "Front-end (React, Typescript), software engineering " \
+ "(ideally Python), and experience with LLMs."
eval_persona = f"You are responsible to finding good recruiting " \
+ "candidates, for the company description. " \
+ f"Ideal canddiates have skills: {skills}. " \
+ "Submit your candidate evaluation with the submit_evaluation tool. "


eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tool_ids=[read_resume_tool.id, submit_evaluation_tool.id]
)

#### Outreach agent 

In [13]:
def email_candidate(content: str): 
    """
    Send an email

    Args: 
        content (str): Content of the email 
    """
    print("Pretend to email:", content)
    return

email_candidate_tool = client.create_tool(email_candidate)

In [14]:
outreach_persona = "You are responsible for sending outbound emails " \
+ "on behalf of a company with the send_emails tool to " \
+ "potential candidates. " \
+ "If possible, make sure to personalize the email by appealing " \
+ "to the recipient with details about the company. " \
+ "You position is `Head Recruiter`, and you go by the name Bob, with contact info bob@gmail.com. " \
+ """
Follow this email template: 

Hi <candidate name>, 

<content> 

Best, 
<your name> 
<company name> 
"""

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tool_ids=[email_candidate_tool.id]
)

In [15]:
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message="Candidate: Tony Stark"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 11, 9, 997757, tzinfo=datetime.timezone.utc) updated_at=None id='message-1d598232-fcbc-41da-98a0-7f848941477d' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="The user provided a candidate name. Let's fetch the resume for evaluation.")] organization_id=None agent_id='agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_Er0iPs2AN2owfqDALrsiMOOi', function=Function(arguments='{\n  "name": "Tony Stark",\n  "request_heartbeat": true\n}', name='read_resume'), type='function')] tool_call_id=None step_id=None
Letta.agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce - INFO - Request to call function read_resume with 

LettaResponse(messages=[ReasoningMessage(id='message-1d598232-fcbc-41da-98a0-7f848941477d', date=datetime.datetime(2025, 2, 12, 19, 11, 9, 997757, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="The user provided a candidate name. Let's fetch the resume for evaluation."), ToolCallMessage(id='message-1d598232-fcbc-41da-98a0-7f848941477d', date=datetime.datetime(2025, 2, 12, 19, 11, 9, 997757, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='read_resume', arguments='{\n  "name": "Tony Stark",\n  "request_heartbeat": true\n}', tool_call_id='call_Er0iPs2AN2owfqDALrsiMOOi')), ToolReturnMessage(id='message-e1135968-2d04-4252-8310-a3ad0a5e5489', date=datetime.datetime(2025, 2, 12, 19, 11, 10, 40186, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "Tony Stark\\nFrontend Engineer - React Specialist\\nBoston, MA | (123) 456-7890 | tony.stark@email.com | LinkedIn: /in/tonystark\\n\\nSummary\\nInnovative Frontend Engineer with a Bachelor’s degree in Computer Science from the Massachusetts Institute of Technology (MIT) and over 6 years of professional experience in building and optimizing user-focused web applications. Adept at using React to create efficient, scalable, and dynamic client-side applications.\\n\\nEducation\\nMassachusetts Institute of Technology (MIT)\\nBachelor of Science in Computer Science\\nGraduated: 2018\\n\\nGPA: 4.0\\nDean’s List; Received the MIT Undergraduate Research Opportunities Program (UROP) grant.\\nProfessional Experience\\nSenior Frontend Developer\\nInnovatech Solutions, Boston, MA\\nJune 2020 - Present\\n\\nLead a team of five developers in the design and implementation of a React-based web application for a leading financial services firm, improving user engagement by 40%.\\nImplemented advanced features in React, including Hooks, Context API, and Redux for state management, leading to a 30% reduction in loading times.\\nSpearheaded the migration from AngularJS to React, including updating the architecture and optimizing legacy codebases.\\nFrontend Developer\\nTechGiant Inc, Boston, MA\\nJuly 2018 - May 2020\\n\\nDeveloped new user-facing features using React.js and integrated applications with backend services via RESTful APIs.\\nCollaborated with UX/UI designers to translate designs and wireframes into high-quality code.\\nOptimized components for maximum performance across a vast array of web-capable devices and browsers.\\nConducted comprehensive debugging using React Developer Tools.\\nSkills\\nLanguages: JavaScript (ES6+), HTML5, CSS3, TypeScript\\nFrameworks/Libraries: React, Redux, Next.js, Node.js\\nTools: Webpack, Babel, NPM, Jest, Enzyme\\nOthers: Responsive Design, RESTful Services, Agile Methodologies\\nProjects\\nReal-Time Analytics Dashboard\\nFrontend Developer\\n\\nDeveloped a real-time analytics dashboard using React and Redux, which supported WebSocket for real-time updates.\\nImplemented various D3.js visualizations integrated within the React environment for dynamic data presentation.\\nE-Commerce Platform Revamp\\nSenior Frontend Developer\\n\\nLed the frontend development of a full-scale revamp for a major e-commerce platform, improving the site\'s performance and user experience.\\nIntegrated Stripe for payment processing and managed state with Redux-Saga.\\nCertifications\\nCertified React Developer - React Training Institute, 2019\\nAgile Certified Practitioner (PMI-ACP) – Project Management Institute, 2020\\nInterests\\nAugmented Reality Development\\nOpen Source Contribution\\nRobotics\\nTony\'s background and experience are designed to showcase a strong skill set in React development and an impressive academic background from MIT.\\n",\n  "time": "2025-02-12 02:11:10 PM EST-0500"\n}', status='success', tool_call_id='call_Er0iPs2AN2owfqDALrsiMOOi', stdout=None, stderr=None), ReasoningMessage(id='message-c3ac2ec1-d988-4c51-8d9f-e295a15b8122',

In [16]:
feedback = "Our company pivoted to foundation model training"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 11, 46, 17385, tzinfo=datetime.timezone.utc) updated_at=None id='message-5b2c56d9-a596-4481-91e9-f95d6a8eb35c' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="The company's focus has shifted to foundation model training. I must update my core memory to reflect this.")] organization_id=None agent_id='agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_ZCiasxKV50xLGC21k0IJo3Yt', function=Function(arguments='{\n  "label": "Company Description",\n  "old_content": "The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.",\n  "new_content": "The company, Ag

In [17]:
feedback = "The company is also renamed to FoundationAI"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 11, 51, 137094, tzinfo=datetime.timezone.utc) updated_at=None id='message-d0d8db51-5a29-47d9-b71a-e1a349dbc823' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="The company's name has changed to FoundationAI. I need to update my core memory with this information.")] organization_id=None agent_id='agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_JBjZJ066CO9lLuS8SmkbNrhl', function=Function(arguments='{\n  "label": "Company Description",\n  "old_content": "The company, AgentOS, has pivoted its focus to foundation model training.",\n  "new_content": "The company, now renamed to FoundationAI, has pivot

In [18]:
response

LettaResponse(messages=[ReasoningMessage(id='message-d0d8db51-5a29-47d9-b71a-e1a349dbc823', date=datetime.datetime(2025, 2, 12, 19, 11, 51, 137094, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="The company's name has changed to FoundationAI. I need to update my core memory with this information."), ToolCallMessage(id='message-d0d8db51-5a29-47d9-b71a-e1a349dbc823', date=datetime.datetime(2025, 2, 12, 19, 11, 51, 137094, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_replace', arguments='{\n  "label": "Company Description",\n  "old_content": "The company, AgentOS, has pivoted its focus to foundation model training.",\n  "new_content": "The company, now renamed to FoundationAI, has pivoted its focus to foundation model training.",\n  "request_heartbeat": false\n}', tool_call_id='call_JBjZJ066CO9lLuS8SmkbNrhl')), ToolReturnMessage(id='message-f422c222-eeb4-4b10-bc9e-df8994d0dcc5', date=datetime.datetime(2025, 2, 12, 19, 11, 51, 231247, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 02:11:51 PM EST-0500"\n}', status='success', tool_call_id='call_JBjZJ066CO9lLuS8SmkbNrhl', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=100, prompt_tokens=4095, total_tokens=4195, step_count=1))

In [19]:
response = client.send_message(
    agent_name="eval_agent", 
    role="system", 
    message="Candidate: Spongebob Squarepants"
)

Letta.agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce - WARNING - Warning: Attempting to run ChatCompletion without user as the last message in the queue
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 11, 54, 673114, tzinfo=datetime.timezone.utc) updated_at=None id='message-8e1fa55c-6a57-4a18-aeb7-ed1735dbdd1b' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='A new candidate, Spongebob Squarepants, has been alerted. Time to fetch the resume for evaluation.')] organization_id=None agent_id='agent-f41d547e-a51d-45df-96ff-eac4d76cc4ce' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_anTYFIWQBhIMjABDUt30Z3JB', function=Function(arguments='{\n  "name": "Spongebob Squarepants",\n  "request_heartbe

In [20]:
response

LettaResponse(messages=[ReasoningMessage(id='message-8e1fa55c-6a57-4a18-aeb7-ed1735dbdd1b', date=datetime.datetime(2025, 2, 12, 19, 11, 54, 673114, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='A new candidate, Spongebob Squarepants, has been alerted. Time to fetch the resume for evaluation.'), ToolCallMessage(id='message-8e1fa55c-6a57-4a18-aeb7-ed1735dbdd1b', date=datetime.datetime(2025, 2, 12, 19, 11, 54, 673114, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='read_resume', arguments='{\n  "name": "Spongebob Squarepants",\n  "request_heartbeat": true\n}', tool_call_id='call_anTYFIWQBhIMjABDUt30Z3JB')), ToolReturnMessage(id='message-e297e221-5a7d-4094-973c-0f818a499c81', date=datetime.datetime(2025, 2, 12, 19, 11, 54, 699683, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "Spongebob Squarepants\\nAI Researcher - Agent Technology Specialist\\nPhoenix, AZ | (321) 654-9870 | spongebob.squarepants@email.com | LinkedIn: /in/spongebobsquarepants\\n\\nSummary\\nEnthusiastic and innovative AI Researcher with a Ph.D. in Computer Science, specializing in agent technology. With over 8 years of experience in academic and applied AI research, I am passionate about developing scalable AI solutions that improve decision-making processes and user interactions in autonomous systems.\\n\\nEducation\\nUniversity of Arizona, School of Information\\nPh.D. in Computer Science - Specialization in Artificial Intelligence\\nGraduated: 2016\\n\\nDissertation: \\"Enhancing Multi-Agent Systems for Real-Time Adaptive Learning\\"\\nAwarded the University\'s Distinguished Research Fellowship.\\nArizona State University\\nBachelor of Science in Computer Science\\nGraduated: 2012\\n\\nGraduated Summa Cum Laude.\\nActive member of the AI Club and ACM student chapter.\\nProfessional Experience\\nSenior AI Researcher\\nFutureTech Labs, Phoenix, AZ\\nAugust 2016 - Present\\n\\nLead a team of researchers focusing on the development of advanced agent-based models for AI systems used in logistics and transportation.\\nPublished over 15 papers in top-tier journals and conferences on topics such as agent adaptability, machine learning, and distributed AI systems.\\nDeveloped a proprietary framework for multi-agent systems that increased the efficiency of real-time data processing by 50%.\\nAI Research Intern\\nInnovative AI Solutions, Tucson, AZ\\nJune 2015 - August 2015\\n\\nAssisted in the development of scalable AI algorithms that are now patented and used in predictive modeling and decision support tools.\\nPresented findings at the National AI Conference, receiving accolades for \\"Best Use of AI in Predictive Analysis\\".\\nSkills\\nProgramming Languages: Python, Java, C++\\nAI Technologies: Machine Learning, Deep Learning, Reinforcement Learning, Multi-Agent Systems\\nTools/Platforms: TensorFlow, PyTorch, Scikit-learn, Keras, Jupyter\\nData Handling: Big Data Technologies, SQL, NoSQL\\nPublications and Conferences\\nSquarepants, S. \\"Agent-Based Models for Real-Time Adaptive Learning,\\" Journal of Artificial Intelligence Research, 2019.\\nSquarepants, S., et al. \\"Distributed Learning in Multi-Agent Environments,\\" Proceedings of the International Conference on Autonomous Agents, 2021.\\nCertifications\\nCertified Data Scientist - Data Science Council of America (DASCA), 2018\\nMachine Learning Specialist Certification - Coursera, 2017\\nInterests\\nDeveloping educational tools using AI to enhance learning in underserved communities.\\nEnvironmental conservation projects using AI technologies.\\nThis resume builds a comprehensive profile for Spongebob as an expert in AI, particularly in agent technologies, with a strong educational and professional background in the Southwest.\\n",\n  "time": "2025-02-12 02:11:54 PM EST-0500"\n}', status='success', tool_call_id='call_anTYFIWQBhIMjABDUt30Z3JB', stdout=None, stderr=None)

In [21]:
client.get_core_memory(outreach_agent.id).get_blocks()

[Block(value='The company, now renamed to FoundationAI, has pivoted its focus to foundation model training.', limit=5000, template_name='company', is_template=False, label='Company Description', description=None, metadata={}, id='block-43df646d-5cf2-411a-90f6-ed6ab855d4db', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Block(value='You are responsible for sending outbound emails on behalf of a company with the send_emails tool to potential candidates. If possible, make sure to personalize the email by appealing to the recipient with details about the company. You position is `Head Recruiter`, and you go by the name Bob, with contact info bob@gmail.com. \nFollow this email template: \n\nHi <candidate name>, \n\n<content> \n\nBest, \n<your name> \n<company name> \n', limit=5000, template_name='persona', is_template=False, label='Agent Persona', description=None, metadata={}, id='block-d41b9866-bb1e-4ecd-a399-e0e463275119', orga

## Section 3: Adding an orchestrator agent 

In [22]:
# delete and re-create agents 
for agent in client.list_agents():
    client.delete_agent(agent.id)

eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tool_ids=[read_resume_tool.id, submit_evaluation_tool.id]
)

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tool_ids=[email_candidate_tool.id]
)

In [23]:
client.get_block(org_block.id)

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=5000, template_name='company', is_template=False, label='Company Description', description=None, metadata={}, id='block-43df646d-5cf2-411a-90f6-ed6ab855d4db', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

In [24]:
from typing import Optional

def search_candidates_db(page: int) -> Optional[str]: 
    """
    Returns 1 candidates per page. 
    Must start at page 0.
    Page 0 returns the first 1 candidate, 
    Page 1 returns the next 1, etc.
    Returns `None` if no candidates remain. 

    Args: 
        page (int): The page number to return candidates from 

    Returns: 
        candidate_names (List[str]): Names of the candidates
    """
    
    names = ["Tony Stark", "Spongebob Squarepants", "Gautam Fang"]
    if page >= len(names): 
        return None
    return names[page]

def consider_candidate(name: str): 
    """
    Submit a candidate for consideration. 

    Args: 
        name (str): Candidate name to consider 
    """
    from letta import create_client 
    client = create_client()
    message = f"Consider candidate {name}" 
    print("Sending message to eval agent: ", message)
    response = client.send_message(
        agent_name="eval_agent", 
        role="user", 
        message=message
    ) 


In [25]:
search_candidate_tool = client.create_tool(search_candidates_db)
consider_candidate_tool = client.create_tool(consider_candidate)

# create recruiter agent
recruiter_agent = client.create_agent(
    name="recruiter_agent", 
    memory=OrgMemory(
        persona="You run a recruiting process for a company. " \
        + "Your job is to continue to pull candidates from the " 
        + "`search_candidates_db` tool until there are no more " \
        + "candidates left. " \
        + "For each candidate, consider the candidate by calling "
        + "the `consider_candidate` tool. " \
        + "You should continue to call `search_candidates_db` " \
        + "followed by `consider_candidate` until there are no more " \
        " candidates. Start at page 0. ",
        org_block=org_block
    ), 
    tool_ids=[search_candidate_tool.id, consider_candidate_tool.id]
)
   

In [26]:
response = client.send_message(
    agent_id=recruiter_agent.id, 
    role="system", 
    message="Run generation"
)

Letta.agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd - WARNING - Warning: Attempting to run ChatCompletion without user as the last message in the queue
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 13, 56, 499977, tzinfo=datetime.timezone.utc) updated_at=None id='message-3cd03992-363f-4eef-bcb9-700cffb91fcd' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='The user has logged in. Time to start the recruitment process.')] organization_id=None agent_id='agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_y1R3Jq49oh5QVwmq7xAOHqhH', function=Function(arguments='{\n  "message": "Welcome! Let\'s start the recruitment process. I\'ll be pulling candidates f

In [27]:
response

LettaResponse(messages=[ReasoningMessage(id='message-3cd03992-363f-4eef-bcb9-700cffb91fcd', date=datetime.datetime(2025, 2, 12, 19, 13, 56, 499977, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='The user has logged in. Time to start the recruitment process.'), ToolCallMessage(id='message-3cd03992-363f-4eef-bcb9-700cffb91fcd', date=datetime.datetime(2025, 2, 12, 19, 13, 56, 499977, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Welcome! Let\'s start the recruitment process. I\'ll be pulling candidates from our database for consideration."\n}', tool_call_id='call_y1R3Jq49oh5QVwmq7xAOHqhH')), ToolReturnMessage(id='message-6684bbae-b2c5-4cd8-b419-4e7b069be400', date=datetime.datetime(2025, 2, 12, 19, 13, 56, 503614, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 02:13:56 PM EST-0500"\n}', status='success', tool_call_id='call_y1R3Jq49oh5QVwmq7xAOHqhH', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=53, prompt_tokens=2309, total_tokens=2362, step_count=1))

In [28]:
response = client.send_message(
    agent_id=recruiter_agent.id, 
    role="system", 
    message="Complete your recruiting process please."
)

Letta.agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd - WARNING - Warning: Attempting to run ChatCompletion without user as the last message in the queue
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 14, 31, 286668, tzinfo=datetime.timezone.utc) updated_at=None id='message-7dc50d12-9c0f-4093-ba9f-3cf2e7fbc4c4' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='Initiating candidate search from the database. Starting from page 0.')] organization_id=None agent_id='agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_TYlDiibWemJr4bFhKpCyoB9z', function=Function(arguments='{\n  "page": 0,\n  "request_heartbeat": true\n}', name='search_candidates_db'), type='f

In [29]:
response 

LettaResponse(messages=[ReasoningMessage(id='message-7dc50d12-9c0f-4093-ba9f-3cf2e7fbc4c4', date=datetime.datetime(2025, 2, 12, 19, 14, 31, 286668, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Initiating candidate search from the database. Starting from page 0.'), ToolCallMessage(id='message-7dc50d12-9c0f-4093-ba9f-3cf2e7fbc4c4', date=datetime.datetime(2025, 2, 12, 19, 14, 31, 286668, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='search_candidates_db', arguments='{\n  "page": 0,\n  "request_heartbeat": true\n}', tool_call_id='call_TYlDiibWemJr4bFhKpCyoB9z')), ToolReturnMessage(id='message-3244d641-2c5a-4136-820d-d7c67f40c61e', date=datetime.datetime(2025, 2, 12, 19, 14, 31, 307312, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "Tony Stark",\n  "time": "2025-02-12 02:14:31 PM EST-0500"\n}', status='success', tool_call_id='call_TYlDiibWemJr4bFhKpCyoB9z', stdout=None, stderr=None), ReasoningMessage(id='message-7592f5cc-551e-4423-8293-8220b5e53673', date=datetime.datetime(2025, 2, 12, 19, 14, 34, 364958, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="We have a candidate named Tony Stark. Let's consider him for the position."), ToolCallMessage(id='message-7592f5cc-551e-4423-8293-8220b5e53673', date=datetime.datetime(2025, 2, 12, 19, 14, 34, 364958, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='consider_candidate', arguments='{\n  "name": "Tony Stark",\n  "request_heartbeat": true\n}', tool_call_id='call_umnWhdUNPzP7g21FCQRQ1L2c')), ToolReturnMessage(id='message-269ee965-6ce4-43fe-8446-af68a6f3ecac', date=datetime.datetime(2025, 2, 12, 19, 15, 24, 765147, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 02:15:24 PM EST-0500"\n}', status='success', tool_call_id='call_umnWhdUNPzP7g21FCQRQ1L2c', stdout=None, stderr=None), ReasoningMessage(id='message-2c366993-aff3-42f7-9bfc-1b8b56336a5c', date=datetime.datetime(2025, 2, 12, 19, 15, 27, 408022, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Tony Stark has been considered for the position. Time to pull the next candidate.'), ToolCallMessage(id='message-2c366993-aff3-42f7-9bfc-1b8b56336a5c', date=datetime.datetime(2025, 2, 12, 19, 15, 27, 408022, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Tony Stark has been considered for the position. Moving on to the next candidate."\n}', tool_call_id='call_RP4CGMz9p5bvajdCxKuEIwL8')), ToolReturnMessage(id='message-0f9b6ef9-9252-4dac-a6f2-ddc20f12bd18', date=datetime.datetime(2025, 2, 12, 19, 15, 27, 413827, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 02:15:27 PM EST-0500"\n}', status='success', tool_call_id='call_RP4CGMz9p5bvajdCxKuEIwL8', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=145, prompt_tokens=7834, total_tokens=7979, step_count=3))

In [30]:
response = client.send_message(
    agent_id=recruiter_agent.id, 
    role="system", 
    message="Complete your recruiting process please."
)
response 

Letta.agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd - WARNING - Warning: Attempting to run ChatCompletion without user as the last message in the queue
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 19, 16, 18, 9522, tzinfo=datetime.timezone.utc) updated_at=None id='message-7ed8a323-a0f6-4910-b55a-f6ae63d35f20' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='Continuing the candidate search from the database. Moving on to page 1.')] organization_id=None agent_id='agent-faa944a1-51a2-4c82-8df1-ad3f62fd1ccd' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_IkVUDxf4tA0nDEt2upa3W2By', function=Function(arguments='{\n  "page": 1,\n  "request_heartbeat": true\n}', name='search_candidates_db'), type='

LettaResponse(messages=[ReasoningMessage(id='message-7ed8a323-a0f6-4910-b55a-f6ae63d35f20', date=datetime.datetime(2025, 2, 12, 19, 16, 18, 9522, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Continuing the candidate search from the database. Moving on to page 1.'), ToolCallMessage(id='message-7ed8a323-a0f6-4910-b55a-f6ae63d35f20', date=datetime.datetime(2025, 2, 12, 19, 16, 18, 9522, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='search_candidates_db', arguments='{\n  "page": 1,\n  "request_heartbeat": true\n}', tool_call_id='call_IkVUDxf4tA0nDEt2upa3W2By')), ToolReturnMessage(id='message-6308040b-1fe7-42c6-9935-e64672c5b3b5', date=datetime.datetime(2025, 2, 12, 19, 16, 18, 34194, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "Spongebob Squarepants",\n  "time": "2025-02-12 02:16:18 PM EST-0500"\n}', status='success', tool_call_id='call_IkVUDxf4tA0nDEt2upa3W2By', stdout=None, stderr=None), ReasoningMessage(id='message-4bb14b42-9938-40f5-a50f-3fedd6142e1a', date=datetime.datetime(2025, 2, 12, 19, 16, 21, 231285, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="We have a candidate named Spongebob Squarepants. Let's consider him for the position."), ToolCallMessage(id='message-4bb14b42-9938-40f5-a50f-3fedd6142e1a', date=datetime.datetime(2025, 2, 12, 19, 16, 21, 231285, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='consider_candidate', arguments='{\n  "name": "Spongebob Squarepants",\n  "request_heartbeat": true\n}', tool_call_id='call_HtuBPSaylazsLIPZ55wEgRTU')), ToolReturnMessage(id='message-10635b5b-fb70-4152-8419-5d87f3730f55', date=datetime.datetime(2025, 2, 12, 19, 16, 57, 434580, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 02:16:57 PM EST-0500"\n}', status='success', tool_call_id='call_HtuBPSaylazsLIPZ55wEgRTU', stdout=None, stderr=None), ReasoningMessage(id='message-c412c559-81c5-4fe8-8980-9ec0103cc620', date=datetime.datetime(2025, 2, 12, 19, 17, 0, 596182, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Spongebob Squarepants has been considered for the position. Time to pull the next candidate.'), ToolCallMessage(id='message-c412c559-81c5-4fe8-8980-9ec0103cc620', date=datetime.datetime(2025, 2, 12, 19, 17, 0, 596182, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Spongebob Squarepants has been considered for the position. Moving on to the next candidate."\n}', tool_call_id='call_vwupN7OOsNZTKI0LH8BEGUWg')), ToolReturnMessage(id='message-04c1e879-f048-4cb8-882b-ec34d192b74e', date=datetime.datetime(2025, 2, 12, 19, 17, 0, 603273, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 02:17:00 PM EST-0500"\n}', status='success', tool_call_id='call_vwupN7OOsNZTKI0LH8BEGUWg', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=158, prompt_tokens=9229, total_tokens=9387, step_count=3))